Import libraries:

In [10]:
from __future__ import unicode_literals
from nltk.tokenize import word_tokenize
import wikipediaapi
import string

import fuzzy_pandas as fpd

import pandas as pd
from collections import Counter
from webdriver_manager.chrome import ChromeDriverManager
from dateutil.relativedelta import relativedelta
from datetime import datetime as dt
    
import os
from tqdm import tqdm
from docx import Document
from docx.shared import Cm
from docx.shared import Pt
import concurrent.futures
from flask import Flask, render_template, request
from docx.enum.dml import MSO_THEME_COLOR_INDEX
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.shared import Pt
import docx
import time

from reportlab.lib.styles import ParagraphStyle
from reportlab.platypus import SimpleDocTemplate, Paragraph
from reportlab.lib.enums import TA_JUSTIFY, TA_CENTER
from reportlab.platypus import Table, TableStyle
from reportlab.lib.pagesizes import letter
from reportlab.lib.units import inch
from reportlab.platypus import Image
from reportlab import platypus

from sumy.parsers.html import HtmlParser
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer as Summarizer
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words

import numpy as np
import pandas as pd
from tika import parser
from gensim.summarization.summarizer import summarize

import requests

from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO

from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud
from requests import get
import random
import html
import re

import nltk
import collections
nltk.download('punkt')
from textblob import TextBlob
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

from urllib.request import urlopen
from bs4 import BeautifulSoup
from bs4.element import Comment

import networkx as nx
from nltk.tokenize import sent_tokenize

import spacy
import neuralcoref
from spacy.symbols import nsubj, nsubjpass, VERB

from selenium.webdriver.common.keys import Keys
import chromedriver_binary

from selenium import webdriver
from bs4 import BeautifulSoup
from bs4.element import Tag
from random import randint
import winsound

nlp = spacy.load('en_core_web_lg')
nlp.max_length = 50000000

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\skamenshchikov\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Define CDFs:

In [11]:
def striphtml(data):
    p = re.compile(r'<.*?>')
    return p.sub('', data)

def get_unique_text(document):
    unique_sentences = []
    for sentence in [sent.raw for sent in TextBlob(document).sentences]:
        if sentence not in unique_sentences:
            unique_sentences.append(sentence)
    return ' '.join(unique_sentences)

def getText(filename):
    doc = docx.Document(filename)
    fullText = []
    for para in doc.paragraphs:
        fullText.append(para.text)
    return '\n'.join(fullText)

def text_normalize(txt):
    processed_text = (re.sub('[^a-zA-Z]', ' ', txt)).lower()
    processed_text=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",processed_text)
    processed_text=re.sub("(\\d|\\W)+"," ",processed_text)

    stop_words = set(stopwords.words("english"))
    lemmatizer = WordNetLemmatizer()
    tokens = word_tokenize(processed_text)
    tokens = [lemmatizer.lemmatize(word) for word in tokens if not word in stop_words]
    tokens = [i for i in tokens if (tags(i) in ['NN', 'NNP', 'NNS', 'NNPS'])]

    return tokens

def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True

def text_from_html(body):
    soup = BeautifulSoup(body, 'html.parser')
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)
    return u" ".join(t.strip() for t in visible_texts)

def connected_component_subgraphs(G):
    for c in nx.connected_components(G):
        yield G.subgraph(c)

def get_text(url):
    page = urlopen(url)
    soup = BeautifulSoup(page)
    fetched_text = ' '.join(map(lambda p:p.text,soup.find_all('p')))
    return fetched_text

def create_graph(text, common):
    tokens = text_normalize(text)
    bigrams=list(nltk.ngrams(tokens, 2))

    bigram_counts = collections.Counter(bigrams)
    bigram_df = pd.DataFrame(bigram_counts.most_common(common), columns=['bigram', 'count'])
    d = bigram_df.set_index('bigram').T.to_dict('records')

    F = nx.Graph()
    for k, v in d[0].items():
            F.add_edge(k[0], k[1], weight=(v*10))
            pos = nx.spring_layout(F, iterations=500)
    return F, pos

def graph_keys(final_text, top_number):
    F,pos = create_graph(final_text.lower(), top_number)
    nodes = []
    degree = []

    for i in F.nodes():
        nodes.append(i)
        degree.append(F.degree(i))

    x = dict(zip(nodes, degree))
    key_nodes = list({k: v for k, v in sorted(x.items(), key=lambda item: item[1], reverse=True)})
    tokens = text_normalize(final_text.lower())

    bigrams=list(nltk.ngrams(tokens, 2))
    bigram_counts = collections.Counter(bigrams)
    bigram_df = pd.DataFrame(bigram_counts.most_common(100), columns=['bigram', 'count'])
    key_bigrams = [' '.join(i) for i in list(bigram_df['bigram'])]

    keys = [i for i in key_bigrams if (((i.split()[0] in key_nodes) or (i.split()[1] in key_nodes)) and (i.split()[0]!=i.split()[1]))][:top_number]
    key_bigrams = ', '.join(keys)

    return key_bigrams

def tags(x):
    return nltk.pos_tag(nltk.word_tokenize(x))[0][1]

def syntax_full(spacy_sentence):
    result=[]
    for token in spacy_sentence:
        if (token.dep == nsubj or token.dep == nsubjpass) and token.head.pos == VERB:
            result.append(token.head)
    if result:
        return True
    else:
        return False

def check_min_num_of_clauses(spacy_sentence, n):
    result=[]
    for token in spacy_sentence:
        if (token.dep_ in ['nsubj','nsubjpass','csubj','expl']) and (token.head.pos_ == 'VERB' or token.head.pos_ == 'AUX'):
            result.append(token.head.text)
    if len(result)>=n:
        return True

    else:
        return False

def get_sents_ids_whitelist(spacy_sents):
    whitelist=[]
    i=1
    sents_texts=[]
    for sent in spacy_sents:
        if (sent.text not in sents_texts) and check_min_num_of_clauses(sent,1):
            whitelist.append(i)
            sents_texts.append(sent.text)
        i=i+1
    return(whitelist)

def get_list_sents_tokens(spacy_sents,sents_whitelist,blacklist_tokens):
    sents_tokens=[]
    n=1
    for sent in spacy_sents:
        sent_tokens=[]
        if (n in sents_whitelist):
            for token in sent:
                if (token.i not in blacklist_tokens):
                    sent_tokens.append(token.text)
            sents_tokens.append(sent_tokens)
            sent_tokens=[]

        n=n+1
    return(sents_tokens)

def detokenizer(list_of_tokens):
    text_str="".join([" "+w if not w.startswith("'") and not w.startswith("’") and w!='' and w not in string.punctuation else w for w in list_of_tokens]).strip()
    return(text_str)

def sentence_grammar_fix(sentences):
    fixed=[]
    for sent in sentences:

        sent=sent.strip()
        sent=sent.replace('\n','')
        sent=sent.replace('()','')

        sent=re.sub('\s+',' ',sent)
        sent=sent+'.'
        sent=re.sub(r'([,.\-—:])+',r'\1',sent)

        if len(sent)>1:
            if sent[0] in ['.',',','-','—']:
                sent=sent[1:]
        sent=sent.strip()

        if len(sent)>1:
            if sent[0].isalpha():
                sent=sent[0].upper()+sent[1:]
        fixed.append(sent)

    return(fixed)

def parse_arxiv(query, D1, D2): 
    arxivtext = ''
    urls = []
    titles = []

    closest_value = 200
    
    req = 'https://arxiv.org/search/?query='+query+'&size='+str(closest_value)+'&searchtype=all&source=header&start=0'
    req = req + '&date-from_date=' + str(D1) + 'date-to_date=' + str(D2) + '&order=-announced_date_first' 
    htmlString = get(req)

    soup = BeautifulSoup(htmlString.content, 'html5lib')
    hrefs = soup.find_all('a', {'href': re.compile(r'arxiv.org/abs/')})

    titles = list(soup.find_all('p', {'class' : 'title is-5 mathjax'}))[:page_number]
    titles_r = [i.text.replace('\n','').replace('  ','') for i in titles]
    titles = ', '.join(titles_r)

    if (len(hrefs) > 0):
        for i in hrefs:
            urls.append(i['href'])

    txt = []
    for i in urls[:page_number]:
        time.sleep(random.randint(1,8))
        soup = BeautifulSoup(get(str(i)).content, 'html5lib')
        abstract = ' '.join(soup.find('blockquote').text.replace('  ',' ').split())
        txt.append(abstract)

    arxivtext = re.sub('[^A-Za-z0-9.]+', ' ', '; '.join(txt))
    df = pd.DataFrame(list(zip(txt, urls, titles_r)), columns=['text','link', 'page'])

    return df

def parse_page(url,tag,cls=''): 
    
    htmlString = get(url).text
    soup = BeautifulSoup(htmlString, 'html.parser')
    paragraphs = soup.find(tag,cls)
    txt = text_from_html(str(paragraphs))

    return txt

def get_entities(rawtext, entities_number):
    spacy_nlp = spacy.load('en_core_web_lg', disable=["tagger","parser"])
    doc = spacy_nlp(rawtext)

    ners = []
    for ent in doc.ents:
        if ent.label_ in ['ORG', 'PERSON']:
            ners.append(ent.text)

    ners = dict(Counter(ners))
    ners = sorted(ners.items(), key=lambda x: x[1], reverse=True)
    ners = list(frozenset([i[0] for i in ners][:entities_number]))

    return ners

def google_urls(query, page_number):   
    
    # load driver   
    driver_location = "chrome/chromedriver.exe"
    options = webdriver.ChromeOptions()
    options.add_argument('--lang=en')
    options.add_argument("--incognito")
    options.add_argument("--enable-javascript")
    driver = webdriver.Chrome(executable_path=driver_location, chrome_options=options)
    #/load driver

    # get urls
    google_url = "https://www.google.com/search?q=" + query + "&num=" + str(page_number+1) + '&lr=lang_en'
    driver.get(google_url)
    time.sleep(randint(1,5))

    soup = BeautifulSoup(driver.page_source,'lxml')
    result_div = soup.find_all('div', attrs={'class': 'g'})

    links = []    
    for r in result_div:
        try:
            link = r.find('a', href=True)
            title = None
            title = r.find('h3')

            if isinstance(title,Tag):
                title = title.get_text()

            description = None
            description = r.find('span', attrs={'class': 'st'})

            if isinstance(description, Tag):
                description = description.get_text()

            if link != '' and title != '' and description != '':
                links.append(link['href'])

        except Exception as e:
            print(e)
            continue

    url_list = list(frozenset(links))[:(page_number)]
    #/ get urls
    
    # stop driver
    driver.stop_client()
    driver.close()
    #/stop driver
    
    return url_list 

def parse_urls(url_list, tag, cls_on, cls=''):
    
    txt = []
    titles = []
    errors = []
    
    # load content 
    for j in tqdm(url_list):   
        
        time.sleep(randint(1,5))
        
        # load driver   
        driver_location = "chrome/chromedriver.exe"
        options = webdriver.ChromeOptions()
        options.add_argument('--lang=en')
        options.add_argument("--incognito")
        options.add_argument("--enable-javascript")
    
        driver = webdriver.Chrome(executable_path=driver_location, chrome_options=options)
        #/load driver
        
        try:  
            if str(j).endswith('.pdf'): 
                file_data = parser.from_file(str(j))           
                t = file_data['content']
            else:   
                # get content
                driver.get(j)
                time.sleep(randint(1,5))
                #/get content

                # filter content
                soup = BeautifulSoup(driver.page_source,'html.parser')
                #/filter content
                 
                if cls_on==True:
                    t = text_from_html(str(soup.find(tag, attrs={'class': cls})))
                else:
                    t = text_from_html(str(soup.find(tag)))
                    
                txt.append(re.sub('[^A-Za-z0-9.]+', ' ', t))
                titles.append(''.join(sent_tokenize(t)[:3]))
        except:
            print('Parsing error:',str(j))
            errors.append(str(j))
            
        # stop driver
        driver.stop_client()
        driver.close()
        #/stop driver
        
        #/load content        
    
    df = pd.DataFrame(list(zip(txt, url_list, titles)), columns=['text','link', 'page'])
    
    return df

Parameters:

In [12]:
UPLOAD_FOLDER = 'docs/'
ALLOWED_EXTENSIONS = set(['pdf', 'docx'])

page_number = 20
keys_number = 20

process_time = 0
delta_year = 1

wiki_FT = True
google_FT = True
arxiv_FT = True
patent_FT = True

Create query:

In [13]:
query = input()

machine learning


Load language model:

In [14]:
spacy_nlp = spacy.load('en_core_web_lg', disable=['tagger','parser'])

Define dates:

In [15]:
D1 = (dt.now() - relativedelta(years=delta_year)).strftime('%Y-%m-%d')
D2 = dt.now().strftime('%Y-%m-%d')

Parse Google - limited:

In [16]:
%%time
google_time = 0
start = time.time()

if google_FT == True: 
    try: 
        # parse data
        url_list = list(set(google_urls((query + " before:" + str(D2) + " after:" + str(D1)), page_number)))
        df_google = parse_urls(url_list, 'body', False)
        #/parse data      
        
        # filter links
        exclusions = ['patents.google.com', 'angel.co']
        df_google = df_google[~df_google['link'].str.contains('|'.join(exclusions))]
        #/filter links
        
        # find ners
        googleners = ', '.join(get_entities('; '.join(list(df_google['text'])), keys_number))
        # find ners
        
        # find keys
        googlekeys = graph_keys('; '.join(list(df_google['text'].apply(lambda x: ', '.join(text_normalize(x))))), keys_number)
        #/find keys
        
        # print data
        print('Keys:', googlekeys)
        print('\nEntities:', googleners, '\n')
        #/print data
        
    except:
        print('No data')
    
    end = time.time()
    google_time = end - start
    
winsound.Beep(2500, 1000)    

C:\Users\skamenshchikov\Anaconda3\lib\site-packages\ipykernel_launcher.py:236: DeprecationWarning: use options instead of chrome_options
100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [04:32<00:00, 13.29s/it]


Keys: data science, machine algorithm, use case, machine model, training data, machine application, data set, data scientist, interview question, learn machine, course machine, decision tree, video lecture, linear regression, computer science, towards data, ml algorithm, machine course, ai machine, specialization course

Entities: Semi, Coursera, IBM, Andrew Ng, AI Course, Big Data, DL, ML, Deutsch Espa, Google, YouTube Stores, Deep, AI, Machine, Machine Learning, MATLAB, RL, NXP, Fran, Artificial Intelligence 

Wall time: 6min 3s


Parse Angel.co - limited:

In [17]:
%%time
arxiv_time = 0
start = time.time()

if arxiv_FT == True:    
    try:
        # get urls
        angel_query = "site:angel.co/company/ " + query + " before:" + str(D2) + " after:" + str(D1)
        url_list = google_urls(angel_query, page_number*10)
        url_list = list(set([('https://angel.co/company/' + i.split('/')[4]) for i in url_list]))
        #/get urls
               
        # parse data
        df_angel = parse_urls(url_list, 'h3', True, 'styles_component__1kg4S styles_flow__3_K06 styles_highConcept__1eLSa __halo_fontSizeMap_size--xl __halo_fontWeight_medium')
        #/parse data  
        
        # filter links
        inclusions = ['angel.co']
        df_angel = df_angel[df_angel['link'].str.contains('|'.join(inclusions))]
        #/filter links
        
        # find ners
        angelners = ', '.join(get_entities('; '.join(list(df_angel['text'])), keys_number))
        # /find ners
        
        # normalize and find keys
        angelkeys = graph_keys('; '.join(list(df_angel['text'].apply(lambda x: ', '.join(text_normalize(x))))), keys_number)
        #/normalize and find keys
        
        print('Keys:', angelkeys)
        print('\nEntities:', angelners, '\n')
        
    except:
        print('No data')
    
    end = time.time()
    arxiv_time = end - start

winsound.Beep(2500, 1000)  

C:\Users\skamenshchikov\Anaconda3\lib\site-packages\ipykernel_launcher.py:236: DeprecationWarning: use options instead of chrome_options
100%|██████████████████████████████████████████████████████████████████████████████████| 91/91 [19:30<00:00, 12.15s/it]


Keys: computer vision, language processing, ai operating, operating system, platform data, ai ml, vision platform, platform use, use deep, deep train, train computer, computer play, play game, game marketing, marketing economy, economy predict, predict order, order pickup, pickup delivery, delivery pizzeria

Entities: AI ML, Natural Language Processing to Automate Routine Work, Climate Informatics, Quality Prediciton in Process Industry, Powering Intelligent Industrial Workspaces, Ecommerce, Making Factories Smarter, The Automation Company, Forestry ML, Deep Ge, Listnr, ML, Simplifying Healthcare for Patients and Doctors, AI and Machine Learning, Deep, Revolutions, AI, Machine Learning, AC, Intelligent Natural Language Processing 

Wall time: 19min 59s


Parse Google Patents - limited: 

In [18]:
%%time
patent_time = 0
start = time.time() 

if patent_FT == True:   
    try:        
        # get urls
        patent_query = "site:https://patents.google.com " + query + " before:" + str(D2) + " after:" + str(D1)
        url_list = list(set(google_urls(patent_query, page_number*10)))
        #/get urls
        
        # parse data
        df_patent = parse_urls(url_list, 'h1', True, 'scroll-target style-scope patent-result')
        #/parse data

        # filter links
        inclusions = ['https://patents.google.com']
        df_patent = df_patent[df_patent['link'].str.contains('|'.join(inclusions))]
        df_patent = df_patent[df_patent['link'].str.contains('/en')]
        #/filter links
        
        # find ners
        patentners = ', '.join(get_entities('; '.join(list(df_patent['text'])), keys_number))
        #/find ners
               
        # normalize and find keys
        patentkeys = graph_keys('; '.join(list(df_patent['text'].apply(lambda x: ', '.join(text_normalize(x))))), keys_number)
        #/normalize and find keys
        
        print('Keys:', patentkeys)
        print('\nEntities:', patentners, '\n')
        
    except:
        print('No data')  
       
    end = time.time()
    patent_time = end - start   
    
winsound.Beep(2500, 1000)

C:\Users\skamenshchikov\Anaconda3\lib\site-packages\ipykernel_launcher.py:236: DeprecationWarning: use options instead of chrome_options
100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [23:17<00:00, 15.16s/it]


Keys: generate link, link comment, system method, method generate, comment method, system generate, comment generate, comment system, comment kind, comment character, data generate, thereof generate, error detection, function generate, pronunciation generate, comment device, risk generate, information processing, comment management, management channel

Entities: Improved PSNR Peak Signal, Pyramid, brca1, LTE, VMOS, Multimodal, ARPA Auto Radar, Shaped, Method, Program, Device, IPTV, Compositions, OCR, Target, Phenotypic, Slotted CSMA CA, System 

Wall time: 23min 43s


Create specific keys:

In [19]:
%%time
specific = (angelkeys + ', '  + patentkeys).split(', ')
known = googlekeys.split(', ')

df1 = pd.DataFrame(list(zip(specific)), columns =['specific'])
df2 = pd.DataFrame(list(zip(known)), columns =['known'])

matches = fpd.fuzzy_merge(df1, df2, left_on=['specific'], right_on=['known'], ignore_case=True, keep='match', 
                         method='levenshtein', threshold=0.8, join='inner')

specific_reqs = list(set(specific).difference(set(list(matches['specific']))))
specific_keys = ', '.join(specific_reqs)

print('Specific keys:', specific_keys, '\n')

winsound.Beep(2500, 1000)

Specific keys: information processing, operating system, predict order, comment management, use deep, order pickup, economy predict, generate link, train computer, language processing, comment generate, comment device, comment method, pronunciation generate, system generate, platform use, risk generate, pickup delivery, vision platform, platform data, delivery pizzeria, thereof generate, error detection, method generate, computer play, play game, marketing economy, function generate, link comment, ai operating, deep train, data generate, management channel, ai ml, system method, computer vision, comment system, comment character, game marketing, comment kind 

Wall time: 1.31 s


Test for direct match:

In [ ]:
%%time
requests = []
volumes = []

# load driver   
driver_location = "chrome/chromedriver.exe"
options = webdriver.ChromeOptions()
options.add_argument('--lang=en')
options.add_argument("--incognito")
options.add_argument("--enable-javascript")
    
driver = webdriver.Chrome(executable_path=driver_location, chrome_options=options)
#/load driver

for i in tqdm(specific_reqs):
    # get urls
    query = i + " before:" + str(D2) + " after:" + str(D1)
    #/get urls
                
    # get content
    google_url = "https://www.google.com/search?q=" + i + "&num=" + str(page_number+1) + '&lr=lang_en'
    driver.get(google_url)
    time.sleep(randint(1,5))
    #/get content

    # filter content
    soup = BeautifulSoup(driver.page_source,'html.parser')
    #/filter content
                 
    txt = text_from_html(str(soup.find('body')))
    txt = re.sub('[^A-Za-z0-9.]+', ' ', txt)
    volumes.append(txt.count(i))
            
# stop driver
driver.stop_client()
driver.close()
#/stop driver      
    
winsound.Beep(2500, 1000)    

C:\Users\skamenshchikov\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: use options instead of chrome_options
  # This is added back by InteractiveShellApp.init_path()
 18%|██████████████▌                                                                    | 7/40 [00:39<03:12,  5.84s/it]

Report dataframe:

In [ ]:
df_report = pd.DataFrame(list(zip(requests, volumes)), columns =['requests','volumes'])
df_report.sort_values(by=['volumes'], ascending=True, inplace=True)
df_report.head(20)